# Video pipeline with Yolo + Resnet Model

In [83]:
import os
import sys
from pprint import PrettyPrinter
import matplotlib.pyplot as plt
from typing import Dict, List
import numpy as np

pp = PrettyPrinter(indent=4)
from barazmoon.twitter import twitter_workload_generator

# get an absolute path to the directory that contains parent files
__file__ = globals()["_dh"][0]
project_dir = __file__ = globals()["_dh"][0]
sys.path.append(os.path.normpath(os.path.join(project_dir, "..", "..", "..")))

from experiments.utils.constants import FINAL_RESULTS_PATH
from experiments.utils.parser import AdaptationParser
from experiments.utils.drawing import draw_temporal, draw_cumulative

In [84]:
metaserieses = [21, 21, 21, 21, 21]
serieses = [1, 21, 41, 61, 81]

series_names = None
# series_names = {
#     # 1: "IPA-accuracy",
#     2: "IPA",
#     # 3: "IPA-cost",
#     4: "FA2-high",
#     # 5: "FA2-high",
#     # 6: "RIM-low",
#     7: "RIM-high",
# }
factors = {
    1: 2/3,
    21: 2,
    41: 2,
    61: 1,
    81: 2/3
}
series_paths = {
    series: os.path.join(
        FINAL_RESULTS_PATH, "metaseries", str(metaseries), "series", str(series)
    )
    for series, metaseries in zip(serieses, metaserieses)
}

loaders = {
    series: AdaptationParser(
        series_path=series_path, model_name="video", type_of="router_pipeline"
    )
    for series, series_path in series_paths.items()
}

In [85]:
simulation_modes = {}
configs = {}
drop_limit = {}
for series, loader in loaders.items():
    configs_exp = loader.load_configs()
    config = configs_exp["0.yaml"]
    configs[series] = config
    simulation_modes[series] = config["simulation_mode"]
    drop_limit[series] = config["drop_limit"]
drop_limit

{1: 10, 21: 20, 41: 20, 61: 5, 81: 20}

In [86]:
# load the sent workload
sent_loads = {}
for series, config in configs.items():
    workload_type = config["workload_type"]
    workload_config = config["workload_config"][0]
    start = workload_config["start"]
    end = workload_config["end"]
    damping_factor = workload_config["damping_factor"]
    sent_loads[series] = twitter_workload_generator(
        days=f"{start}-{end}", damping_factor=damping_factor
    )

In [87]:
adaptation_logs = dict(
    map(lambda l: (l[0], l[1].load_adaptation_log()), loaders.items())
)

In [88]:
slas = {}
stage_wise_slas = {}
series_changes = {}
for series in serieses:
    series_changes[series] = loaders[series].series_changes(
        adaptation_log=adaptation_logs[series]
    )
    slas[series] = series_changes[series]["sla"] / factors[series]
    stage_wise_slas[series] = dict(
        map(
            lambda l: (l[0], l[1] / factors[series]),
            series_changes[series]["stage_wise_slas"].items(),
        )
    )
print(slas)
print(stage_wise_slas)

{1: 6.897365409135817, 21: 9.235554277896881, 41: 10.132449761033056, 61: 3.851757884025572, 81: 17.619139596819878}
{1: {'crop': 4.620585805177688, 'classification': 2.2767796039581296}, 21: {'audio': 8.344102382659912, 'nlp-qa': 0.8914518952369689}, 41: {'audio': 9.043847739696501, 'nlp-sent': 1.0886020213365553}, 61: {'nlp-sum': 2.527679383754729, 'nlp-qa': 1.324078500270843}, 81: {'nlp-li': 0.9724401533603668, 'nlp-trans': 12.855180367827415, 'nlp-sum': 3.7915190756320936}}


In [89]:
sla_and_drop = {}
for series in serieses:
    sla_and_drop[series] = {}
    sla_and_drop[series]["sla"] = slas[series]
    sla_and_drop[series]["stage_wise_slas"] = stage_wise_slas[series]
    sla_and_drop[series]["drop_limit"] = drop_limit[series]
sla_and_drop

{1: {'sla': 6.897365409135817,
  'stage_wise_slas': {'crop': 4.620585805177688,
   'classification': 2.2767796039581296},
  'drop_limit': 10},
 21: {'sla': 9.235554277896881,
  'stage_wise_slas': {'audio': 8.344102382659912,
   'nlp-qa': 0.8914518952369689},
  'drop_limit': 20},
 41: {'sla': 10.132449761033056,
  'stage_wise_slas': {'audio': 9.043847739696501,
   'nlp-sent': 1.0886020213365553},
  'drop_limit': 20},
 61: {'sla': 3.851757884025572,
  'stage_wise_slas': {'nlp-sum': 2.527679383754729,
   'nlp-qa': 1.324078500270843},
  'drop_limit': 5},
 81: {'sla': 17.619139596819878,
  'stage_wise_slas': {'nlp-li': 0.9724401533603668,
   'nlp-trans': 12.855180367827415,
   'nlp-sum': 3.7915190756320936},
  'drop_limit': 20}}

# Real Latencies

## Maeseured p99 Latency


In [90]:
timeout_per_second = {}
per_second_results = {}
for series in serieses:
    if not simulation_modes[series]:
        timeout_per_second[series], per_second_results[series] = loaders[
            series
        ].per_second_result_processing()
    else:
        timeout_per_second[series], per_second_results[series] = None, None

## timeouts

In [91]:
import numpy as np

if not all(simulation_modes.values()):
    ylabel = "% SLA Violations"
    xlabel = "Experiments"
    timeout_dics = {}
    for series in serieses:
        # print(50 * "-" + f" {series} " + 50 * "-")
        if not simulation_modes[series]:
            timeout_dics[series] = {
                "": (
                    np.array(timeout_per_second[series]) / sum(sent_loads[series])
                ).tolist()
            }
            # draw_temporal(timeout_dics[series])
            print(f"{sum(timeout_per_second[series])} out of {sum(sent_loads[series])}")
    draw_temporal(timeout_dics, multiple_experiments=True, ylabel=ylabel)
    draw_cumulative(
        timeout_dics,
        multiple_experiments=True,
        ylabel=ylabel,
        series_names=series_names,
        xlabel=xlabel,
    )